In [18]:
from data_preprocess import data_preprocess, get_training_data, get_input_data, prepare_submission
import pandas as pd
import warnings
import sys

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [19]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
data = data_preprocess(one_hot_location=True)

In [21]:
features = ["direct_rad:W", "direct_rad_1h:J", "clear_sky_rad:W", "clear_sky_energy_1h:J", "diffuse_rad:W", "sun_elevation:d","is_day:idx", 
            "is_in_shadow:idx", "diffuse_rad_1h:J", "t_1000hPa:K","relative_humidity_1000hPa:p", "air_density_2m:kgm3", "A", "B", "C"]

X_train, targets = get_training_data(data, features)

In [85]:
from sklearn import tree

X = X_train.values
y = targets

clf = tree.DecisionTreeRegressor(max_depth=7)
clf = clf.fit(X, y)

In [86]:
predictions = clf.predict(X)
training_mse(targets, predictions)


MSE on training data: 89926.02744998843


In [87]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(clf, X, y, scoring="neg_mean_squared_error", cv=10)
display_cross_val_scores(scores)

MSE scores: [551.85087561 533.80416571 517.42760265 254.93798171 102.36239815
 105.42436811  69.0926574   71.58496534 115.40099748 260.28626395]
Mean MSE: 258.21722761195605
Std. dev: 191.83495466858272


- obviously there is overfitting for this model since it has a MSE that low on the training data, but a much higher one on the test-data

In [33]:
%autoreload

from evaluate import training_mse, display_cross_val_scores

In [18]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(clf, X, y, scoring="neg_mean_squared_error", cv=10)

In [30]:
display_cross_val_scores(scores)
training_mse(targets, predictions, clf)

MSE scores: [716.55854316 707.3528214  691.24196082 327.1586252  126.47676357
 123.69909159  81.59955233 104.42757682 139.2160819  324.9609207 ]
Mean MSE: 334.26919374916235
Std. dev: 255.95616723124724
MSE on training data: 2.2981441193116194e-30


In [31]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(X, y)

RandomForestRegressor()

In [36]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(forest_reg, X, y, scoring="neg_mean_squared_error", cv=10)

In [35]:
predictions = forest_reg.predict(X)

training_mse(targets, predictions)

MSE on training data: 11853.076109338726


In [38]:
display_cross_val_scores(scores)

MSE scores: [529.70669818 506.89103182 497.38989231 241.46934932 100.53897313
 101.08248655  59.10622719  65.47007789 100.62037614 245.15861797]
Mean MSE: 244.7433730484791
Std. dev: 185.08687387460085


- the random forest does not overfit, and does the (sofar) best on the validation sets - even if it does badly on the training set

In [39]:
X_test = get_input_data()
predictions = forest_reg.predict(X_test[features].values)

In [47]:
submission = prepare_submission(X_test, predictions)
submission.to_csv('submissions/random_forest_default_params.csv', index=False)

In [88]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)

LinearSVR(epsilon=1.5)

In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

In [89]:
predictions = svm_reg.predict(X)

training_mse(targets, predictions)

MSE on training data: 843044.0164840152


In [9]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(scaled_X_train.values, scaled_targets)

predictions = svm_reg.predict(scaled_X_train.values)

In [16]:
predictions = targets_scaler.inverse_transform(predictions.reshape(-1, 1))
training_mse(targets, predictions)

MSE on training data: 859395.0884100501


In [10]:
from evaluate import training_mse

training_mse(scaled_targets, predictions)

MSE on training data: 1.4621114256465122


In [11]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(svm_reg, scaled_X_train.values, scaled_targets, scoring="neg_mean_squared_error", cv=10)



NameError: name 'display_cross_val_scores' is not defined

In [12]:
from evaluate import display_cross_val_scores

display_cross_val_scores(scores)

MSE scores: [1.35010613 1.32582788 1.37259458 1.09509979 1.10316355 1.11718991
 1.15668621 1.11523194 1.20535046 1.21088918]
Mean MSE: 1.2052139621181392
Std. dev: 0.10214172121295914


In [7]:
from sklearn.preprocessing import StandardScaler

targets_scaler = StandardScaler()
scaled_targets = targets_scaler.fit_transform(targets.values.reshape(-1, 1))  # Assuming 'y' is your target variable

# to unscale
# predictions_original_scale = targets_scaler.inverse_transform(predictions_scaled)

In [104]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(scaled_X_train.values, targets)

In [ ]:
predictions_scaled = svm_poly_reg.predict(scaled_X_train.values)
training_mse(scaled_targets, predictions_scaled)

In [53]:
#scaling the input features - this is important for KNR

import pandas as pd
from sklearn.preprocessing import StandardScaler

X_test = get_input_data()

scaler = StandardScaler()

num_features = ["direct_rad:W", "direct_rad_1h:J", "clear_sky_rad:W", "clear_sky_energy_1h:J", "diffuse_rad:W", "sun_elevation:d","is_day:idx", 
            "is_in_shadow:idx", "diffuse_rad_1h:J", "t_1000hPa:K","relative_humidity_1000hPa:p", "air_density_2m:kgm3"]

scaled_X_train_num = pd.DataFrame(scaler.fit_transform(X_train[num_features]), columns=num_features)
scaled_X_test_num = pd.DataFrame(scaler.transform(X_test[num_features]), columns=num_features)

In [54]:
scaled_X_train = X_train.copy()
scaled_X_train.update(scaled_X_train_num)

scaled_X_test = X_test.copy()
scaled_X_test.update(scaled_X_test_num)

In [64]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=7)
knn.fit(scaled_X_train, targets)

KNeighborsRegressor(n_neighbors=7)

In [65]:
predictions = knn.predict(scaled_X_train)

In [66]:
from evaluate import training_mse

training_mse(targets, predictions)

MSE on training data: 116551.55739695576


In [67]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(knn, scaled_X_train.values, targets, scoring="neg_mean_squared_error", cv=10)

In [68]:
display_cross_val_scores(scores)

MSE scores: [ 615.23310834  670.85582174  631.97013697  255.28166585  114.65954032
  114.9661818   132.44310654  211.84712149  436.03879907 1246.12488944]
Mean MSE: 442.94203715424646
Std. dev: 341.01016087094575
